In [2]:
!pip install sdv
!pip install sdmetrics

In [3]:
# !unrar x data.rar

In [4]:
import pandas as pd
import numpy as np
import torch
import random
import warnings
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score
from hyperopt import fmin, rand, hp, Trials, STATUS_OK
from hyperopt import space_eval
from sdv.single_table import CTGANSynthesizer as CTGAN
from sdv.metadata import SingleTableMetadata
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import tqdm
from sklearn.metrics import f1_score, r2_score
from xgboost import XGBClassifier, XGBRegressor
from sdmetrics.reports.single_table import QualityReport
import pandas as pd

In [5]:
dir_datasets = '/kaggle/input/nir02-main/data/'

# Загрузка реальных датасетов
real_data_1 = pd.read_csv(dir_datasets+'insurance.csv')
real_data_2 = pd.read_csv(dir_datasets+'my_classification.csv')
real_data_3 = pd.read_csv(dir_datasets+'my_regression.csv')
real_data_4 = pd.read_csv(dir_datasets+'two_moons.csv')
real_data_5 = pd.read_csv(dir_datasets+'california_housing.csv')
real_data_6 = pd.read_csv(dir_datasets+'titanic.csv')

# Словарь датасетов для удобства
datasets = {
        'titanic': {
                        "data": real_data_6,
                        "task": "classification",
                        "target": "Survived",
                        "num_columns": ["Age", "Fare", "SibSp", "Parch"],
                        "cat_columns": ["Pclass", "Sex", "Embarked"]
                    },
        'california_housing': {
                        "data": real_data_5,
                        "task": "regression",
                        "target": "MedHouseVal",
                        "num_columns":
                        ["MedInc",	"HouseAge",	"AveRooms",	"AveBedrms",	"Population",	"AveOccup",	"Latitude",	"Longitude"],
                        "cat_columns":
                        []
                    },
        'insurance': {
                        "data": real_data_1,
                        "task": "regression",
                        "target": "expenses",
                        "num_columns":
                        ["age", "bmi", "children"],
                        "cat_columns":
                        ["sex", "smoker", "region"]
                    }

}

for name, data in datasets.items():
    print(f" Информация о датасете {name}:")
    print(f" Количество строк: {data['data'].shape[0]}")
    print(f" Количество колонок: {data['data'].shape[1]}")
    print(f" Колонки: {list(data['data'].columns)}")
    print(f" Задача: {data['task']}")
    print(f" Целевая переменная: {data['target']}")
    print(f" Числовые признаки: {data['num_columns']}")
    print(f" Категориальные признаки: {data['cat_columns']}")
    print()

 Информация о датасете titanic:
 Количество строк: 712
 Количество колонок: 12
 Колонки: ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
 Задача: classification
 Целевая переменная: Survived
 Числовые признаки: ['Age', 'Fare', 'SibSp', 'Parch']
 Категориальные признаки: ['Pclass', 'Sex', 'Embarked']

 Информация о датасете california_housing:
 Количество строк: 1200
 Количество колонок: 9
 Колонки: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude', 'MedHouseVal']
 Задача: regression
 Целевая переменная: MedHouseVal
 Числовые признаки: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
 Категориальные признаки: []

 Информация о датасете insurance:
 Количество строк: 1338
 Количество колонок: 7
 Колонки: ['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'expenses']
 Задача: regression
 Целевая переменная: expenses
 Число

In [6]:
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [7]:
def process_data(dataset, num_columns, cat_columns, transformation_num_type='None', transformation_cat_type='None'):

    df_processed = dataset.copy()

    # Обработка числовых признаков
    if transformation_num_type == 'CDF':
        # CDF трансформация: преобразует значения в их эмпирическую функцию распределения
        for col in num_columns:
            # Правильная формула для эмпирической CDF
            df_processed[col] = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

    elif transformation_num_type == 'PLE_CDF':
        # PLE_CDF (Probability Logit Envelope CDF) - более сложная трансформация
        for col in num_columns:
            # Сначала применяем CDF
            cdf_values = (df_processed[col].rank(method='average') - 0.5) / len(df_processed)

            # Затем применяем logit трансформацию с небольшим сглаживанием
            # Избегаем 0 и 1 для предотвращения бесконечности в logit
            epsilon = 1e-6
            cdf_values = np.clip(cdf_values, epsilon, 1 - epsilon)

            # Логит трансформация: ln(p/(1-p))
            df_processed[col] = np.log(cdf_values / (1 - cdf_values))

    # Обработка категориальных признаков
    if transformation_cat_type == 'OHE':
        # One Hot Encoding для категориальных признаков
        for col in cat_columns:
            # Создаем dummy переменные с префиксом имени колонки
            dummy_df = pd.get_dummies(df_processed[col], prefix=col, dtype=int)

            # Удаляем исходную категориальную колонку
            df_processed = df_processed.drop(columns=[col])

            # Добавляем новые dummy колонки
            df_processed = pd.concat([df_processed, dummy_df], axis=1)

    return df_processed

In [8]:
ctgan_space = {
    # Learning rates
    'discriminator_lr': hp.qloguniform('discriminator_lr',
                                                 np.log(4e-4), np.log(2.1e-3), 5e-5),
    'generator_lr': hp.qloguniform('generator_lr',
                                             np.log(5e-5), np.log(5e-3), 5e-5),

    # Batch size
    'batch_size': hp.choice('batch_size', [100, 500, 1000]),

    # Embedding dimensions
    'embedding_dim': hp.choice('embedding_dim', [32, 128]),

    # Network architecture
    'generator_dim': hp.choice('generator_dim', [[128, 128, 128], [128, 128, 128, 128]]),
    'discriminator_dim': hp.choice('discriminator_dim', [[256, 256], [256, 256, 256]]),

    # Decay parameters
    'generator_decay': hp.qloguniform('generator_decay',
                                     np.log(1e-6), np.log(6.4e-6), 1e-7),
    'discriminator_decay': hp.qloguniform('discriminator_decay',
                                         np.log(1e-6), np.log(8e-6), 1e-6),

    # Frequency
    'log_frequency': hp.choice('log_frequency', [False, True]),
    'transformation_num_type': hp.choice('transformation_num_type', ['CDF', 'PLE_CDF']),
    'transformation_cat_type': hp.choice('transformation_cat_type', ['OHE'])

}

In [9]:
def evaluate_shape(real_data, synthetic_data, cat_columns):
    """
    Прямое использование метрик SDMetrics без QualityReport.
    Для случаев, когда QualityReport не работает.
    """
    # Обработка входных данных
    if hasattr(synthetic_data, "dataframe"):
        synthetic = synthetic_data.dataframe()
    else:
        synthetic = synthetic_data.copy()

    real_data = pd.get_dummies(real_data, columns=cat_columns, drop_first=True)
    synthetic = pd.get_dummies(synthetic, columns=cat_columns, drop_first=True)

    # После get_dummies
    real_data = real_data.astype(float)
    synthetic = synthetic.astype(float)

    # Пытаемся использовать прямые метрики
    shape_scores = []

    from sdmetrics.single_column import KSComplement

    with warnings.catch_warnings():
        warnings.simplefilter('ignore', RuntimeWarning)
        for column in real_data.select_dtypes(include=['number']).columns:
            if column in synthetic.columns:
                ks_score = KSComplement.compute(
                    real_data=real_data[column],
                    synthetic_data=synthetic[column]
                )
                shape_scores.append(ks_score)

    return sum(shape_scores) / len(shape_scores) if shape_scores else 0.0

In [10]:
import sys
import os
import builtins
import contextlib

@contextlib.contextmanager
def suppress_all_output():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        old_print = builtins.print
        builtins.print = lambda *args, **kwargs: None
        sys.stdout = devnull
        sys.stderr = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr
            builtins.print = old_print

In [11]:
from sklearn.model_selection import KFold

def generate_and_evaluate_shape_scores(params, dataset_info, n_splits=3):
    warnings.filterwarnings("ignore", category=FutureWarning)
    warnings.filterwarnings("ignore", category=UserWarning)

    columns_dataset = dataset_info['num_columns'] + dataset_info['cat_columns'] + [dataset_info['target']]
    data = process_data(dataset_info['data'][columns_dataset], dataset_info['num_columns'], dataset_info['cat_columns'],
                        'None', 'None')

    kf = KFold(n_splits=n_splits, shuffle=False)
    shape_scores = []

    for train_index, test_index in kf.split(data):
        train_data = data.iloc[train_index].reset_index(drop=True)
        test_data = data.iloc[test_index].reset_index(drop=True)

        metadata = SingleTableMetadata()
        metadata.detect_from_dataframe(data=train_data)

        # Создание и обучение CTGAN с заданными параметрами
        ctgan = CTGAN(
            metadata=metadata,
            discriminator_lr=params['discriminator_lr'],
            generator_lr=params['generator_lr'],
            batch_size=params['batch_size'],
            embedding_dim=params['embedding_dim'],
            generator_dim=params['generator_dim'],
            generator_decay=params['generator_decay'],
            discriminator_decay=params['discriminator_decay'],
            log_frequency=params['log_frequency'],
        )

        ctgan.fit(train_data)

        synthetic_data = ctgan.sample(len(test_data))

        cat_columns = dataset_info['cat_columns'].copy()
        if dataset_info["task"] == 'classification':
            cat_columns.append(dataset_info['target'])

        score = evaluate_shape(
            test_data,
            synthetic_data,
            cat_columns)

        shape_scores.append(score)

    return np.mean(shape_scores)

In [12]:
def optimize_dataset(dataset_name, dataset_info, max_evals=30):
    print(f"Оптимизация для датасета: {dataset_name}")

    # Создаем объект для хранения истории поиска
    trials = Trials()

    # Определяем целевую функцию для оптимизации
    def objective(params):
        print("="*50)
        shape_score = generate_and_evaluate_shape_scores(params, dataset_info)

        print(f"Shape-score: {shape_score}")
        print(params)
        print("="*50)

        # Для Shape-score мы хотим максимизировать метрику, поэтому возвращаем отрицательное значение
        return {
            'loss': -shape_score,  # отрицательное для максимизации
            'status': STATUS_OK,
            'shape_score': shape_score
        }

    # Запускаем оптимизацию с помощью Rand алгоритма
    rng = np.random.default_rng(RANDOM_SEED)  # Используем фиксированный seed
    best = fmin(
        fn=objective,
        space=ctgan_space,
        algo=rand.suggest,
        max_evals=max_evals,
        trials=trials,
        rstate=rng
    )

    # Автоматическое декодирование параметров
    best_params = space_eval(ctgan_space, best)

    # Находим лучший результат
    best_trial = min(trials.trials, key=lambda x: x['result']['loss'])
    best_shape_score = best_trial['result'].get('shape_score', None)

    results = {
        'best_params': best_params,
        'best_shape_score': best_shape_score,
        'shape_score_diff_from_optimal': -best_trial['result']['loss']  # убираем минус для читаемости
    }

    print(f"Лучший Shape-score для {dataset_name}: {best_shape_score}")
    print(f"Лучшие параметры: {best_params}")
    print('-' * 50)

    return results

In [13]:
# Словарь для хранения результатов
all_results = {}

# Указываем количество итераций для каждого датасета
max_evals = 50

# Запускаем оптимизацию для каждого датасета
for dataset_name in datasets:
    data = datasets[dataset_name]
    all_results[dataset_name] = optimize_dataset(dataset_name, data, max_evals)

Оптимизация для датасета: titanic
Shape-score: 0.8806196740299495                       
{'batch_size': 100, 'discriminator_decay': 4e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.0014500000000000001, 'embedding_dim': 128, 'generator_decay': 3.3999999999999996e-06, 'generator_dim': (128, 128, 128), 'generator_lr': 0.0008500000000000001, 'log_frequency': True, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'CDF'}
Shape-score: 0.8931319363188314                                                    
{'batch_size': 100, 'discriminator_decay': 6e-06, 'discriminator_dim': (256, 256, 256), 'discriminator_lr': 0.001, 'embedding_dim': 128, 'generator_decay': 1.1e-06, 'generator_dim': (128, 128, 128, 128), 'generator_lr': 0.0008, 'log_frequency': False, 'transformation_cat_type': 'OHE', 'transformation_num_type': 'PLE_CDF'}
Shape-score: 0.9289218641042915                                                    
{'batch_size': 1000, 'discriminator_decay': 6e-06, 'discrim

In [14]:
for dataset_name in datasets:
    print(dataset_name, all_results[dataset_name]['best_shape_score'])

titanic 0.9369369688803793
california_housing 0.7521296296296297
insurance 0.9515861152632453
